In [23]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

In [24]:
reform = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_additional_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)



In [25]:
situation_california = {
  "people": {
    "you": {
      "age": {
        "2026": 64
      }
    },
    "your partner": {
      "age": {
        "2026": 62
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "you",
        "your partner"
      ],
      "state_name": {
        "2026": "CA"
      },
      "county_fips": {
        "2026": "06069"
      }
    }
  },
  "marital_units": {
    "your marital unit": {
      "members": [
        "you",
        "your partner"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 400000
      }
    ]
  ]
}


In [26]:
simulation_california = Simulation(
    situation=situation_california,
)
reformed_simulation_california = Simulation(
    situation=situation_california,
    reform=reform,
)

In [27]:
import copy
import pandas as pd
from policyengine_us import Simulation

PERIOD = 2026

# ------------------------------------------------------------------
# Helper: get the tax unit's 2026 FPG from the situation
# ------------------------------------------------------------------
def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year (first tax unit)."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Ensure income isn't interfering (FPG shouldn't depend on income, but be safe)
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0

    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

# ------------------------------------------------------------------
# 1) Convenience: run a one-income California couple simulation
# ------------------------------------------------------------------
def calc_ptc_for_income(base_situation: dict, income: float, *, use_reform=False, period=PERIOD):
    """
    Return ACA PTC (household-level) for the given income and year.
    """
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Split income 50/50 between the two adults
    for person in ["you", "your partner"]:
        sit["people"][person]["employment_income"] = {str(period): income / 2}

    sim = Simulation(situation=sit, reform=reform if use_reform else None)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# ------------------------------------------------------------------
# 2) Build targets from model-derived FPG and compute PTCs
# ------------------------------------------------------------------
fpg_2026 = get_tax_unit_fpg(situation_california, period=PERIOD)

percent_targets = {
    "138 % FPL": 1.38,
    "300 % FPL": 3.00,
    "400 % FPL": 4.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline":   calc_ptc_for_income(situation_california, inc, use_reform=False, period=PERIOD),
        "ptc_ira_reform": calc_ptc_for_income(situation_california, inc, use_reform=True,  period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df

,income_label,income_usd,ptc_baseline,ptc_ira_reform
0,"138 % FPL ($29,897.10)",29897.10,0.000000,0.000000
1,"300 % FPL ($64,993.70)",64993.70,36241.796875,38633.562500
2,"400 % FPL ($86,658.27)",86658.27,0.000000,33737.371094


In [28]:
# Get household-level values for California
household_income_california = simulation_california.calculate("employment_income", map_to="household", period=2026)
baseline_california_per_capita_chip = simulation_california.calculate("per_capita_chip", map_to="household", period=2026)
baseline_california_aca_ptc = simulation_california.calculate("aca_ptc", map_to="household", period=2026)
baseline_california_medicaid_cost = simulation_california.calculate("medicaid_cost", map_to="household", period=2026)
baseline_california_net_income_including_health_benefits = simulation_california.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_california_slcsp = simulation_california.calculate("slcsp", map_to="household", period=2026)

reform_california_per_capita_chip = reformed_simulation_california.calculate("per_capita_chip", map_to="household", period=2026)
reform_california_aca_ptc = reformed_simulation_california.calculate("aca_ptc", map_to="household", period=2026)
reform_california_medicaid_cost = reformed_simulation_california.calculate("medicaid_cost", map_to="household", period=2026)
reform_california_net_income_including_health_benefits = reformed_simulation_california.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

# Calculate total benefits for each scenario
baseline_california_total = [sum(x) for x in zip(baseline_california_per_capita_chip, baseline_california_aca_ptc, baseline_california_medicaid_cost)]
reform_california_total = [sum(x) for x in zip(reform_california_per_capita_chip, reform_california_aca_ptc, reform_california_medicaid_cost)]


In [29]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"

In [30]:
# Create California graph
fig_california = go.Figure()

# Add baseline traces (solid lines)
fig_california.add_trace(go.Scatter(
    x=household_income_california, 
    y=baseline_california_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

# Add reform traces (dotted lines)
fig_california.add_trace(go.Scatter(
    x=household_income_california, 
    y=reform_california_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Reform)', 
    line=dict(color=BLUE_PRIMARY, width=2, dash='dot')
))

# Add total lines
fig_california.add_trace(go.Scatter(
    x=household_income_california, 
    y=baseline_california_total, 
    mode='lines', 
    name='Total Benefits (Baseline)', 
    line=dict(color=DARK_GRAY, width=2)
))

fig_california.add_trace(go.Scatter(
    x=household_income_california, 
    y=reform_california_total, 
    mode='lines', 
    name='Total Benefits (Reform)', 
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Update layout
fig_california.update_layout(
    title='California Household (Couple, ages 64 & 62) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 400000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_california = format_fig(fig_california)
fig_california.show()

In [31]:
#Household net income graphs
import plotly.graph_objects as go

# ---------- California couple ----------
fig_ca = go.Figure()

# Baseline (solid)
fig_ca.add_trace(go.Scatter(
    x=household_income_california,
    y=baseline_california_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

# Reform (dotted)
fig_ca.add_trace(go.Scatter(
    x=household_income_california,
    y=reform_california_net_income_including_health_benefits,
    mode='lines',
    name='Health Net Income (Reform)',
    line=dict(color=DARK_GRAY, width=2, dash='dot')
))

# Layout
fig_ca.update_layout(
    title='California Household (Couple, ages 64 & 62) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_ca = format_fig(fig_ca)
fig_ca.show()

# --- Δ Health-adjusted net income (Reform – Baseline) ---
delta_ca = (
    reform_california_net_income_including_health_benefits
    - baseline_california_net_income_including_health_benefits
)

fig_delta_ca = go.Figure()

fig_delta_ca.add_trace(go.Scatter(
    x=household_income_california,
    y=delta_ca,
    mode='lines',
    name='Δ Net Income (Reform – Baseline)',
    line=dict(color=DARK_GRAY, width=2)
))

fig_delta_ca.update_layout(
    title='California Household (Couple, ages 64 & 62) – Impact of Extending Enhanced Premium Tax Credits',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 400_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_ca = format_fig(fig_delta_ca)
fig_delta_ca.show()

In [32]:
# ========================================================================
# MTR CALCULATION FOR CALIFORNIA HOUSEHOLD
# Uses axes-based simulation for net income, then calculates MTR manually
# ========================================================================

# Step 1: Create situation with axes for vectorized net income calculation
situation_ca_for_mtr = {
    "people": {
        "you": {
            "age": {"2026": 64},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "your partner": {
            "age": {"2026": 62},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["you", "your partner"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["you", "your partner"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["you", "your partner"]
        }
    },
    "households": {
        "your household": {
            "members": ["you", "your partner"],
            "state_name": {"2026": "CA"},
            "county_fips": {"2026": "06069"}
            
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 200_000,
            "count": 200,
            "period": "2026"
        }
    ]]
}

# Step 2: Calculate baseline net income using axes
sim_ca_baseline = Simulation(situation=situation_ca_for_mtr)
household_income_ca_mtr = sim_ca_baseline.calculate("employment_income", map_to="household", period=2026)
baseline_ca_net_income = sim_ca_baseline.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3: Calculate reform net income using axes
sim_ca_reform = Simulation(situation=situation_ca_for_mtr, reform=reform)
reform_ca_net_income = sim_ca_reform.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 4: Calculate MTR from adjacent points
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_ca_mtr_incomes, baseline_ca_mtrs = calc_mtr(household_income_ca_mtr, baseline_ca_net_income)
reform_ca_mtr_incomes, reform_ca_mtrs = calc_mtr(household_income_ca_mtr, reform_ca_net_income)

# Step 5: Create the chart
fig_california_mtr = go.Figure()

fig_california_mtr.add_trace(go.Scatter(
    x=baseline_ca_mtr_incomes,
    y=np.clip(baseline_ca_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=DARK_GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_california_mtr.add_trace(go.Scatter(
    x=reform_ca_mtr_incomes,
    y=np.clip(reform_ca_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=BLUE_PRIMARY, width=2, dash='dash'),
    hovertemplate='Income: $%{x:,.0f}<br>Reform MTR: %{y:.1%}<extra></extra>'
))

fig_california_mtr.update_layout(
    title='Marginal tax rate including health benefits - California couple (ages 64 & 62)',
    xaxis_title='Employment income',
    yaxis_title='Marginal tax rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 200_000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_california_mtr = format_fig(fig_california_mtr)
fig_california_mtr.show()